In [ ]:
from helpers import *

In [ ]:
%matplotlib inline

In [ ]:
def eval_case(case):
    ###############################   Load patient #############################
    patient = all_slices(case, pix=True)
    #Get the middle slice
    num_layers = len(patient)
    mid_layer = num_layers//2
    mid_slice = patient[mid_layer]
    ###############################  Fix high contrast #############################
    patient = fix_contrast(np.array(patient))
    mid_slice = patient[mid_layer]
    #########################  Find area that changes most #########################
    mid_slice_stfd = spatial_temporal_finite_difference(mid_slice)
    ###############################  Threshold cleanup #############################
    mid_slice_stfd = threshold_std(mid_slice_stfd,1.)
    #################################  Get Centroid ################################
    # blur
    result = cv2.GaussianBlur(mid_slice_stfd, (9, 9), 0)
    # get centroid
    row,col = get_centroid(result)
    center = line_mask(result, row, col)
    #plt.imshow(center+mid_slice_stfd)
    ###############################  Bounding Box ##################################
    # get roi
    rstart, rend, cstart, cend = bounding_square(result, row, col)
    width = rend-rstart

    mid_slice = map(lambda x: x[rstart:rend, cstart:cend], mid_slice)
    old_mid_slice_stfd = mid_slice_stfd.copy()
    mid_slice_stfd = mid_slice_stfd[rstart:rend,cstart:cend]
    ###############################  Get slice components ##########################
    components = map(lambda i: filtered_components(mid_slice[i]), range(30))
    masks,areas,centroids = map(np.array, zip(*components))
    ###########################  Track components over time #########################
    # match components over time
    tracks = track_objects(masks)

    ###############################  Plot Area Curves #############################
    rareas = np.zeros_like(tracks)
    for i in range(30):
        rareas[:,i] = areas[i][tracks[:,i].astype(int)-1]
    rareas = rareas.T[:-1,:]

    return rareas, best_harmonic_fit(rareas.T)

In [ ]:
cd '/Users/omakhlouf/Desktop/Data Science Bowl/areacurves'

In [ ]:
broken = 0
for i in range(1,101):
    if i%20 == 0:
        print 'Evaluating patient ', i, '...'
    try:
        curve, best = eval_case(i)
        fig = plt.figure(figsize=(30,8))
        plt.subplot(1,2,1)
        plt.plot(curve)
        plt.subplot(1,2,2)
        plt.plot(curve[:,best])
        plt.savefig(str(i))
        plt.close(fig)
    except Exception as e:
        broken += 1
        
print broken

## ++++++++++++++++++++++ EXPERIMENTS +++++++++++++++++++++++++++

## +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++